In [1]:
#change display for the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [1]:
import requests
import newspaper
from newspaper import Article
from newspaper import Source
import pandas as pd
import nltk
import numpy as np

## Getting All Articles

In [2]:
all_links = pd.read_excel('all articles.xlsx', header = None)

In [3]:
all_links.columns = ['links']

In [4]:
all_links.tail()

,links
271,https://www.snaxshot.com/p/half-a-billion-cred...
272,https://www.snaxshot.com/p/yassified-walmart
273,https://www.snaxshot.com/p/foxtrot-markets-com...
274,https://www.snaxshot.com/p/for-the-love-of-chains
275,https://www.snaxshot.com/p/mexican-erewhon


In [5]:
article_links = all_links.loc[all_links.links.str.contains(r'snaxshot.com/p/') & ~all_links.links.str.contains(r'comments')]

In [6]:
#we want to remove the first ever article as it doesn't really contain any content
article_links = article_links.drop(index = 29)

In [7]:
article_links.to_csv('art_links.csv', index = False)

## Testing an Article with Newspaper

In [10]:
article_links = pd.read_csv('art_links.csv')

In [11]:
article_links

,links
0,https://www.snaxshot.com/p/aesthet-o-scope
1,https://www.snaxshot.com/p/brave-new-snax-world
2,https://www.snaxshot.com/p/coca-cola-stole-chr...
3,https://www.snaxshot.com/p/edible-celebrations
4,https://www.snaxshot.com/p/edible-cogitations
...,...
148,https://www.snaxshot.com/p/half-a-billion-cred...
149,https://www.snaxshot.com/p/yassified-walmart
150,https://www.snaxshot.com/p/foxtrot-markets-com...
151,https://www.snaxshot.com/p/for-the-love-of-chains


In [14]:
article_links.links[29]

'https://www.snaxshot.com/p/snax-grams'

In [12]:
url = article_links.links[0]
article = Article(url)

In [13]:
article.download()

In [14]:
article.parse()

In [15]:
article.text

'A newsletter on upcoming food and beverage trends that offers a curation of brands and aesthetics written by Andrea Hernández.\n\nSnaxshot is ad-free as we are community funded, if you enjoy our content, contribute here . 🤗'

In [16]:
article.nlp()

In [17]:
article.keywords

['food',
 'newsletter',
 'hernándezsnaxshot',
 'offers',
 'written',
 'upcoming',
 'funded',
 'trends',
 'curation',
 'enjoy',
 'aesthetoscope']

In [18]:
article.summary

'A newsletter on upcoming food and beverage trends that offers a curation of brands and aesthetics written by Andrea Hernández.\nSnaxshot is ad-free as we are community funded, if you enjoy our content, contribute here .'

In [19]:
article.images

{'https://substackcdn.com/image/fetch/e_trim:10:white/e_trim:10:transparent/h_72,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fb798366c-f0e1-493d-837d-4e3c39b988d7_1920x1080.png',
 'https://substackcdn.com/image/fetch/w_120,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F69587589-79cc-4c4b-aa88-0318d5d098e3_1920x1080.png',
 'https://substackcdn.com/image/fetch/w_1200,h_600,c_fill,f_jpg,q_auto:good,fl_progressive:steep,g_auto/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F69587589-79cc-4c4b-aa88-0318d5d098e3_1920x1080.png',
 'https://substackcdn.com/image/fetch/w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F4abad2e0-3f7f-4f87-be50-5883fcb21ae7_1920x1080.png',
 'htt

In [21]:
from newspaper import fulltext

html = requests.get(url).text
text = fulltext(html)

In [22]:
text

'A newsletter on upcoming food and beverage trends that offers a curation of brands and aesthetics written by Andrea Hernández.\n\nSnaxshot is ad-free as we are community funded, if you enjoy our content, contribute here . 🤗'

I wasn't able to get newspaper to bypass the log in and get the full article.

## Selenium Instead

In [8]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

### Trying to Loop
Headless didn't work in this case?

In [16]:
#options = Options()
#options.add_argument("--headless")  # Run the browser in headless mode (without GUI)

# Initialize the Chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install()) #options = options if you want non-default

# Navigate to the login page
login_url = r"https://email.mg2.substack.com/c/redacted"
driver.get(login_url)

time.sleep(2)

# Find and click the button before entering the password
#button = driver.find_element_by_class_name("login-option.substack-login__login-option") # no longer necessary
#button.click()

# Give the browser some time to process the button click
time.sleep(2)

# Find the username and password input fields and enter your credentials
#username_field = driver.find_element(By.NAME, "email")
#password_field = driver.find_element(By.NAME, "password")

#username_field.send_keys(username)
#password_field.send_keys(password)

# Submit the login form
#password_field.send_keys(Keys.ENTER)

# Give the browser some time to process the login request
time.sleep(3)

titles = []
subtitles = []
dates = []
images = []
whole_texts = []
#start the loop of urls

for i in range(len(article_links)):
    driver.get(article_links.links[i])
    
    time.sleep(0.5)

#Find element
    title = driver.find_element(By.CLASS_NAME, "post-title.unpublished")
    
    sub = driver.find_element(By.CLASS_NAME, "subtitle")
    
    try: #tries one location
        date = driver.find_element(By.XPATH, "//*[@id='main']/div[2]/div/div[1]/div/article/div[2]/div/div[1]/div/div[2]/div[2]/div[1]")
    except: #tries another location if the first fails
        date = driver.find_element(By.XPATH, "//*[@id='main']/div[2]/div/div[1]/div/article/div[2]/div/div[1]/div/div/div/div[1]")
    
    image = driver.find_elements_by_xpath("//a//img")
    imageNames = []
    
    for element in image:
        imageNames.append(element.get_attribute("src"))
        
    whole_text = driver.find_element_by_xpath("//*[@id='main']/div[2]/div/div[1]/div/article/div[4]/div[1]")
    
    #creating the entry for the article
    titles.append(title.text.strip())
    subtitles.append(sub.text.strip())
    dates.append(date.text.strip())
    images.append(imageNames[2:-2])
    whole_texts.append(whole_text.text.strip())

In [18]:
whole_texts[100]

"We’ve separated our “Spoonful of News” section to be available to those who have a free subscription. From now on all in-depth pieces are for our paid subscribers only.\nNew Age Escapism—In efforts to decriminalize magic mushrooms, California Senate gave final approval to a bill legalizing certain psychedelics for people who are 21 or older.\nIf Governor Newsom signs the bill, it would go into effect in 2025 and make it legal to possess or grow plant-based psychedelics, including psychedelic mushrooms.\nIn Los Angeles, weed dispensaries have been openly selling magic mushroom products all throughout this year.\nWe deep dived into the growing movement of psychedelic-adjacent snack and beverages, in the form of legal psychoactives like kava and kanna.\nSnaxshot #40: Legal Psychedelics\nSNAXSHOT\n·\nJANUARY 14, 2022\n🤗 🔮 Peek into the future: Psilly rabbit, drug trips are just for INC.\nRead full story\nFresh off the news that Meati raised a $50M Series C extension—they are laying off 20

## Exporting Data Out

In [19]:
df = pd.DataFrame(list(zip(titles, subtitles, dates, images, whole_texts)), columns =['title', 'subtitle', 'date', 'images', 'text'])

In [20]:
df.to_csv('full_snax_05-24.csv')